In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df_train = pd.read_csv('../../data/kaggle/1056lab-credit-card-fraud-detection/train.csv')
df_test = pd.read_csv('../../data/kaggle/1056lab-credit-card-fraud-detection/test.csv')

In [3]:
df_test.head()

,ID,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,198366,132459,2.079192,0.052662,-1.755167,0.418802,0.310472,-0.886053,0.083326,-0.149153,...,-0.250318,-0.385564,-1.074903,0.351592,0.441535,-0.300612,0.179993,-0.072265,-0.034296,0.99
1,198367,132460,1.596563,-1.588177,-1.186718,-1.710578,-0.378661,0.906028,-0.796517,0.443712,...,0.016012,0.395564,0.993546,-0.024412,-1.613532,-0.392005,-0.055686,0.033064,-0.042575,183.02
2,198368,132460,2.273454,-1.447207,-0.946779,-1.738063,-1.074172,-0.139592,-1.266807,-0.047025,...,-0.384806,-0.142609,0.098889,0.150236,-0.813565,-0.193346,-0.148293,0.020379,-0.063042,21.00
3,198369,132460,1.987406,0.140840,-2.001286,1.071763,0.717042,-0.880590,0.670095,-0.288313,...,-0.274861,0.158934,0.427937,-0.082711,-0.493515,0.434758,-0.473680,-0.045832,-0.075119,35.00
4,198370,132461,-0.398751,1.370313,-0.715049,-0.867699,1.157188,-0.364107,1.151124,-0.316554,...,0.475877,0.182382,1.126963,-0.183504,0.047153,-0.570891,-0.222038,0.657472,0.278362,0.89


In [5]:
df_train

,ID,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,2,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,3,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,4,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,5,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198360,198361,132457.0,-0.474163,1.438815,-0.816746,0.786791,1.067826,-0.207767,1.251817,0.140304,...,0.125517,0.527323,-0.272449,-1.027952,0.237013,-0.312190,0.304261,0.170698,54.21,0
198361,198362,132457.0,1.501100,-2.329402,-0.028328,-0.579340,-2.269449,-0.208058,-1.307538,0.083778,...,-0.006746,-0.112426,0.103570,-0.060057,-0.973943,1.226613,-0.080066,0.009990,314.90,0
198362,198363,132458.0,1.957960,-0.677669,-0.395253,0.762448,-0.685747,0.201289,-0.854448,0.198542,...,0.199416,0.907851,0.061666,0.730593,0.001207,0.568559,-0.009303,-0.047854,14.39,0
198363,198364,132458.0,-1.135983,-2.150924,-1.078870,-0.922886,-1.936992,0.819695,3.101553,-0.311864,...,0.522862,0.629329,1.597087,-0.658563,0.298377,-0.101644,-0.198980,0.084730,804.00,0


In [7]:
df_train.columns

Index(['ID', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9',
       'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
       'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [45]:
## create a pipeline with a scalar

pipeline = Pipeline(
    [
      #  ('sscalar', StandardScaler()),
        ('gauaian naive bayes', SVC() )
    ]
)

df_train.shape

(198365, 32)

In [4]:
# split 

X_full = df_train.iloc[:,0:31]
y_full = df_train.iloc[:,31]


X_train, X_test, y_train, y_test = train_test_split(X_full, y_full)

In [ ]:
pipeline.fit(X_full,y_full)

y_pred = pipeline.predict(X_test)

/home/jez/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [5]:
mean_squared_error(y_test, y_pred)

NameError: name 'y_pred' is not defined

In [6]:
X_train_10k = X_train.iloc[0:1000]

In [ ]:
sns.pairplot(X_train_10k)